In [ ]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline
gpu_core = 1
import os
import random
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_core)

In [ ]:
!date

In [ ]:
! ls -l '../data/validate/2018-11-10_01-20-15_noup/' | tail

In [ ]:
path = '../data/validate/2018-10-27_02-40-13_noup/'
#path = '../data/distributed/'
#path = '../data/history_selfplays/2018-10-24_08-40-13/'

In [ ]:
gameplays = os.listdir(path)

In [ ]:
fullpath = '{}{}'.format(path,random.choice(gameplays))
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

In [ ]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[:]
imageio.mimsave('../data/gif/{}.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [ ]:
i,len(moves)

In [ ]:
def display_gif(fn):
    from IPython import display 
    return display.HTML('<img src="{}">'.format(fn))

In [ ]:
display_gif('../data/gif/{}.gif'.format(fname[:-4]))

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(board_visualizer.get_board_img(statestr))

# ANALYSIS

In [ ]:
network = resnet.get_model(os.path.join(project_basedir,
    '../data/prepare_weight/2018-10-15_14-20-14_noup'[3:]),
    common.board.create_uci_labels(),GPU_CORE=[gpu_core],FILTERS=128,NUM_RES_LAYERS=7)

In [ ]:
network_player = players.NetworkPlayer(state.currentplayer,network,n_playout=400,dnoise=False,c_puct=5)

In [ ]:
network_player.make_move(state=state,actual_move=False)

In [ ]:
network_player.mcts_policy._root._Q

In [ ]:
childs = network_player.mcts_policy._root._children
probs = []
for move in childs:
    val = childs[move]
    print(move,val._n_visits,val._P,val._Q)
    probs.append(val._n_visits)

In [ ]:
plt.plot(probs)
print(probs)

In [ ]:
plt.plot(probs)

In [ ]:
sum(probs)

In [ ]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [ ]:
!mkdir '../data/mstc_vis'

In [ ]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)

In [ ]:
import sys

print(sys.version)

print(sys.executable)

